In [1]:
import symforce
symforce.set_epsilon_to_symbol()

In [42]:
import numpy as np
from symforce.values import Values

import rosbag2_py
from rclpy.serialization import deserialize_message

In [88]:
bag_path = "../bags/dataset_drive_square_no_vid"
# Copied the following from https://qiita.com/nonanonno/items/8f7bce03953709fd5af9
storage_options = rosbag2_py.StorageOptions(uri=bag_path, storage_id="sqlite3")
converter_options = rosbag2_py.ConverterOptions(input_serialization_format="cdr", output_serialization_format="cdr")

sr = rosbag2_py.SequentialReader()
sr.open(storage_options, converter_options)
sr.set_filter(rosbag2_py.StorageFilter(topics=["/odom", "/detections"]))

[INFO] [1698249767.776140642] [rosbag2_storage]: Opened database '../bags/dataset_drive_square_no_vid/rosbag2_2023_10_25-11_14_04_0.db3' for READ_ONLY.


In [89]:
# Create a dictionary of each topic's message type
# This will be used for deserializing the messages.
import importlib
def get_type_from_str(type_str: str):
    # dynamic load message package
    pkg = importlib.import_module(".".join(type_str.split("/")[:-1]))
    return eval(f"pkg.{type_str.split('/')[-1]}")

msg_type_table = {}
for topic in sr.get_all_topics_and_types():
    msg_type_table[topic.name] = get_type_from_str(topic.type)

In [90]:
# Store the first 50 messages
sr.seek(0)
n_msgs = 100
msgs = [sr.read_next() for i in range(n_msgs)]

[INFO] [1698249770.011219450] [rosbag2_storage]: Opened database '../bags/dataset_drive_square_no_vid/rosbag2_2023_10_25-11_14_04_0.db3' for READ_ONLY.


In [91]:
# Iterate through the first 50 messages store them.
for i, (topic_i, bdata_i, time_i) in enumerate(msgs):
    data_i = deserialize_message(bdata_i, msg_type_table[topic_i])
    print(f"{i}: {topic_i}")
    print("---")

0: /odom
---
1: /odom
---
2: /detections
---
3: /detections
---
4: /detections
---
5: /detections
---
6: /detections
---
7: /detections
---
8: /odom
---
9: /detections
---
10: /detections
---
11: /detections
---
12: /detections
---
13: /detections
---
14: /detections
---
15: /odom
---
16: /detections
---
17: /detections
---
18: /detections
---
19: /detections
---
20: /detections
---
21: /detections
---
22: /odom
---
23: /detections
---
24: /detections
---
25: /detections
---
26: /detections
---
27: /detections
---
28: /detections
---
29: /detections
---
30: /odom
---
31: /detections
---
32: /detections
---
33: /detections
---
34: /detections
---
35: /detections
---
36: /detections
---
37: /odom
---
38: /detections
---
39: /detections
---
40: /detections
---
41: /detections
---
42: /detections
---
43: /detections
---
44: /detections
---
45: /odom
---
46: /detections
---
47: /detections
---
48: /detections
---
49: /detections
---
50: /detections
---
51: /odom
---
52: /detections
---
53: 

In [99]:
# Extract detections and odometry messages
f_deserialize_msg = lambda msg: deserialize_message(msg[1], msg_type_table[msg[0]])
f_deserialize_msg_i = lambda i: f_deserialize_msg(msgs[i])
example_odom_msg = msgs[1]
example_odom_data = f_deserialize_msg(example_odom_msg)

example_detections_msg = msgs[2]
example_detections_data = f_deserialize_msg(example_detections_msg)

print(example_odom_data)
print("---")
print(example_detections_data)

nav_msgs.msg.Odometry(header=std_msgs.msg.Header(stamp=builtin_interfaces.msg.Time(sec=1697911413, nanosec=622994469), frame_id='odom'), child_frame_id='base_footprint', pose=geometry_msgs.msg.PoseWithCovariance(pose=geometry_msgs.msg.Pose(position=geometry_msgs.msg.Point(x=0.002499930899425671, y=2.0161891289617244e-06, z=0.0), orientation=geometry_msgs.msg.Quaternion(x=0.0, y=0.0, z=-0.010080474430627446, w=0.9999491907268356)), covariance=array([1.e-01, 0.e+00, 0.e+00, 0.e+00, 0.e+00, 0.e+00, 0.e+00, 1.e-01,
       0.e+00, 0.e+00, 0.e+00, 0.e+00, 0.e+00, 0.e+00, 1.e-01, 0.e+00,
       0.e+00, 0.e+00, 0.e+00, 0.e+00, 0.e+00, 1.e+05, 0.e+00, 0.e+00,
       0.e+00, 0.e+00, 0.e+00, 0.e+00, 1.e+05, 0.e+00, 0.e+00, 0.e+00,
       0.e+00, 0.e+00, 0.e+00, 1.e+05])), twist=geometry_msgs.msg.TwistWithCovariance(twist=geometry_msgs.msg.Twist(linear=geometry_msgs.msg.Vector3(x=0.0, y=0.0, z=0.0), angular=geometry_msgs.msg.Vector3(x=0.0, y=0.0, z=0.0)), covariance=array([0., 0., 0., 0., 0., 0., 

In [103]:
sr.seek(0)
msgs_all = []
while sr.has_next():
    msgs_all.append(sr.read_next())

[INFO] [1698250197.762902513] [rosbag2_storage]: Opened database '../bags/dataset_drive_square_no_vid/rosbag2_2023_10_25-11_14_04_0.db3' for READ_ONLY.
